In [54]:
## Lab | Making predictions with logistic regression

## In this lab, you will be using the Sakila database of movie rentals.
## In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

## Instructions:
## Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.
## Read the data into a Pandas dataframe.
## Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
## Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.
## Create a logistic regression model to predict this variable from the cleaned data.
## Evaluate the results.

In [55]:
pip install mysql-connector-python

In [56]:
## Import libraries
import mysql.connector
from mysql.connector import connect
from mysql.connector import Error
import getpass       ## https://www.geeksforgeeks.org/getpass-and-getuser-in-python-password-without-echo/

import pandas as pd
import datetime
from datetime import date
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression     ## https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [57]:
## Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.

try:
    # Establishing the connection
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        user='root',
        password = getpass.getpass("Enter password: "),
        database='sakila'
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Successfully connected to MySQL Server version ", db_info)
        
        # Creating a cursor object using the cursor() method
        cursor = connection.cursor()
        
        # SQL query to fetch the necessary data
        query = """
        SELECT
            f.film_id,
            f.title,
            f.release_year,
            f.rating,
            f.length,
            f.rental_rate,
            COUNT(r.rental_id) AS number_of_rentals,
            MAX(r.rental_date) AS last_rental_date
        FROM
            film f
        LEFT JOIN inventory i ON f.film_id = i.film_id
        LEFT JOIN rental r ON i.inventory_id = r.inventory_id
        GROUP BY f.film_id

        """
        
        # Executing the SQL command
        cursor.execute(query)
        
        # Fetching all the rows
        records = cursor.fetchall()
        
        print("Data Retrieved: ")
        for row in records:
            print(row)
        
except Error as e:
    print("Error while connecting to MySQL", e)

Successfully connected to MySQL Server version  8.0.36
Data Retrieved: 
(1, 'ACADEMY DINOSAUR', 2006, 'PG', 86, Decimal('0.99'), 24, datetime.datetime(2024, 5, 16, 21, 44, 19))
(2, 'ACE GOLDFINGER', 2006, 'G', 48, Decimal('4.99'), 7, datetime.datetime(2006, 2, 14, 15, 16, 3))
(3, 'ADAPTATION HOLES', 2006, 'NC-17', 50, Decimal('2.99'), 12, datetime.datetime(2005, 8, 23, 13, 54, 39))
(4, 'AFFAIR PREJUDICE', 2006, 'G', 117, Decimal('2.99'), 23, datetime.datetime(2006, 2, 14, 15, 16, 3))
(5, 'AFRICAN EGG', 2006, 'G', 130, Decimal('2.99'), 12, datetime.datetime(2006, 2, 14, 15, 16, 3))
(6, 'AGENT TRUMAN', 2006, 'PG', 169, Decimal('2.99'), 21, datetime.datetime(2005, 8, 21, 16, 3, 1))
(7, 'AIRPLANE SIERRA', 2006, 'PG-13', 62, Decimal('4.99'), 15, datetime.datetime(2005, 8, 22, 17, 18, 5))
(8, 'AIRPORT POLLOCK', 2006, 'R', 54, Decimal('4.99'), 18, datetime.datetime(2005, 8, 23, 20, 24, 36))
(9, 'ALABAMA DEVIL', 2006, 'PG-13', 114, Decimal('2.99'), 12, datetime.datetime(2005, 8, 23, 14, 26, 51

In [58]:
## Read the data into a Pandas dataframe.

try:
    df = pd.read_sql_query(query, connection)
    display(df.head())
    
except Exception as e:
    print(f"Error: {e}")

C:\Users\rainm\AppData\Local\Temp\ipykernel_20460\3570317981.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,film_id,title,release_year,rating,length,rental_rate,number_of_rentals,last_rental_date
0,1,ACADEMY DINOSAUR,2006,PG,86,0.99,24,2024-05-16 21:44:19
1,2,ACE GOLDFINGER,2006,G,48,4.99,7,2006-02-14 15:16:03
2,3,ADAPTATION HOLES,2006,NC-17,50,2.99,12,2005-08-23 13:54:39
3,4,AFFAIR PREJUDICE,2006,G,117,2.99,23,2006-02-14 15:16:03
4,5,AFRICAN EGG,2006,G,130,2.99,12,2006-02-14 15:16:03


In [59]:
## Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   film_id            1000 non-null   int64         
 1   title              1000 non-null   object        
 2   release_year       1000 non-null   int64         
 3   rating             1000 non-null   object        
 4   length             1000 non-null   int64         
 5   rental_rate        1000 non-null   float64       
 6   number_of_rentals  1000 non-null   int64         
 7   last_rental_date   958 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 62.6+ KB


In [61]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from datetime import datetime

# Assuming df is already defined and loaded with data
# Define numerical and categorical features
numerical_features = ['length', 'rental_rate', 'number_of_rentals']

# Check for non-numeric data and impute missing values
for column in numerical_features:
    if column in df.columns:
        if df[column].dtype not in ['float64', 'int64']:
            print(f"Column {column} contains non-numeric data.")
        else:
            # Impute missing values using median strategy
            imputer = SimpleImputer(strategy='median')
            df[column] = imputer.fit_transform(df[[column]])
    else:
        print(f"Column {column} does not exist in the dataframe.")

# Scale numerical features
available_numerical_features = [col for col in numerical_features if col in df.columns]
if available_numerical_features:
    scaler = StandardScaler()
    df[available_numerical_features] = scaler.fit_transform(df[available_numerical_features])
else:
    print("No numerical features available for scaling.")

# Check if the 'rating' column exists before encoding
if 'rating' in df.columns:
    # Encode categorical variables
    encoder = OneHotEncoder(sparse_output=False)
    rating_encoded = encoder.fit_transform(df[['rating']])
    rating_columns = encoder.get_feature_names_out(['rating'])
    df_encoded = pd.DataFrame(rating_encoded, columns=rating_columns)
    df = pd.concat([df.drop('rating', axis=1), df_encoded], axis=1)
else:
    print("Column 'rating' does not exist in the dataframe.")

# Transform temporal features
if 'last_rental_date' in df.columns:
    df['last_rental_date'] = pd.to_datetime(df['last_rental_date'], errors='coerce')
    df['days_since_last_rental'] = (datetime.now() - df['last_rental_date']).dt.days
    df.drop('last_rental_date', axis=1, inplace=True)
else:
    print("Column 'last_rental_date' does not exist in the dataframe.")

# Display the transformed dataframe
print("Transformed DataFrame:")
print(df.head())


Transformed DataFrame:
   film_id             title  release_year    length  rental_rate  \
0        1  ACADEMY DINOSAUR          2006 -0.724445    -1.209308   
1        2    ACE GOLDFINGER          2006 -1.664897     1.221461   
2        3  ADAPTATION HOLES          2006 -1.615399     0.006077   
3        4  AFFAIR PREJUDICE          2006  0.042766     0.006077   
4        5       AFRICAN EGG          2006  0.364499     0.006077   

   number_of_rentals  rating_G  rating_NC-17  rating_PG  rating_PG-13  \
0           1.082990       0.0           0.0        1.0           0.0   
1          -1.231382       1.0           0.0        0.0           0.0   
2          -0.550684       0.0           1.0        0.0           0.0   
3           0.946850       1.0           0.0        0.0           0.0   
4          -0.550684       1.0           0.0        0.0           0.0   

   rating_R  days_since_last_rental  
0       0.0                    13.0  
1       0.0                  6680.0  
2       0

In [62]:
#checking for NaNs
df.isna().sum()

film_id                    0
title                      0
release_year               0
length                     0
rental_rate                0
number_of_rentals          0
rating_G                   0
rating_NC-17               0
rating_PG                  0
rating_PG-13               0
rating_R                   0
days_since_last_rental    42
dtype: int64

In [63]:
#Imputing missing values with median
median= df['days_since_last_rental'].median()

df['days_since_last_rental'].fillna(median, inplace=True)

In [64]:
display(df.head())
df.isna().sum()

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.082990,0.0,0.0,1.0,0.0,0.0,13.0
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231382,1.0,0.0,0.0,0.0,0.0,6680.0
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550684,0.0,1.0,0.0,0.0,0.0,6855.0
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946850,1.0,0.0,0.0,0.0,0.0,6680.0
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550684,1.0,0.0,0.0,0.0,0.0,6680.0


film_id                   0
title                     0
release_year              0
length                    0
rental_rate               0
number_of_rentals         0
rating_G                  0
rating_NC-17              0
rating_PG                 0
rating_PG-13              0
rating_R                  0
days_since_last_rental    0
dtype: int64

In [65]:
#Imputing missing values with median
median= df['days_since_last_rental'].median()

df['days_since_last_rental'].fillna(median, inplace=True)

In [66]:
# Showing the first few rows of the processed DataFrame to ensure transformations and cleaning were successful
display(df.head())
df.isna().sum()

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.082990,0.0,0.0,1.0,0.0,0.0,13.0
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231382,1.0,0.0,0.0,0.0,0.0,6680.0
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550684,0.0,1.0,0.0,0.0,0.0,6855.0
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946850,1.0,0.0,0.0,0.0,0.0,6680.0
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550684,1.0,0.0,0.0,0.0,0.0,6680.0


film_id                   0
title                     0
release_year              0
length                    0
rental_rate               0
number_of_rentals         0
rating_G                  0
rating_NC-17              0
rating_PG                 0
rating_PG-13              0
rating_R                  0
days_since_last_rental    0
dtype: int64

In [67]:
## Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.


In [68]:
# Calculating the most recent rental date in the dataset
most_recent_rental = df['days_since_last_rental'].min()

# Create a boolean column indicating if a film was rented last month
# Assuming "last month" refers to the past 30 days from the most recent rental
df['rented_last_month'] = df['days_since_last_rental'] <= 30

# Select the relevant columns for the query
query_result = df[['film_id', 'title', 'rented_last_month']]

query_result.head()

,film_id,title,rented_last_month
0,1,ACADEMY DINOSAUR,True
1,2,ACE GOLDFINGER,False
2,3,ADAPTATION HOLES,False
3,4,AFFAIR PREJUDICE,False
4,5,AFRICAN EGG,False


In [69]:
## Create a logistic regression model to predict this variable from the cleaned data.

In [72]:
df.head(5)

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental,rented_last_month
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.082990,0.0,0.0,1.0,0.0,0.0,13.0,True
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231382,1.0,0.0,0.0,0.0,0.0,6680.0,False
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550684,0.0,1.0,0.0,0.0,0.0,6855.0,False
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946850,1.0,0.0,0.0,0.0,0.0,6680.0,False
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550684,1.0,0.0,0.0,0.0,0.0,6680.0,False


In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler

# Preparing the dataset
X = df.drop(columns=['film_id', 'title', 'release_year', 'rented_last_month'])
y = df['rented_last_month']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creating and fitting the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Predicting on the test set
y_pred = log_reg.predict(X_test_scaled)

# Evaluating the model
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

precision, recall, f1, accuracy

c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(1.0, 0.0, 0.0, 1.0)

In [71]:
## Evaluate the results.
    # Noting that we got a resut above showing F1 as "0" (stands for a classification that performs poorly), we are going to try an alternative model below.


In [79]:
df.head(5)

,film_id,title,release_year,length,rental_rate,number_of_rentals,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R,days_since_last_rental,rented_last_month
0,1,ACADEMY DINOSAUR,2006,-0.724445,-1.209308,1.082990,0.0,0.0,1.0,0.0,0.0,13.0,True
1,2,ACE GOLDFINGER,2006,-1.664897,1.221461,-1.231382,1.0,0.0,0.0,0.0,0.0,6680.0,False
2,3,ADAPTATION HOLES,2006,-1.615399,0.006077,-0.550684,0.0,1.0,0.0,0.0,0.0,6855.0,False
3,4,AFFAIR PREJUDICE,2006,0.042766,0.006077,0.946850,1.0,0.0,0.0,0.0,0.0,6680.0,False
4,5,AFRICAN EGG,2006,0.364499,0.006077,-0.550684,1.0,0.0,0.0,0.0,0.0,6680.0,False


In [89]:
df.columns



Index(['film_id', 'title', 'release_year', 'length', 'rental_rate',
       'number_of_rentals', 'rating_G', 'rating_NC-17', 'rating_PG',
       'rating_PG-13', 'rating_R', 'days_since_last_rental',
       'rented_last_month'],
      dtype='object')

In [80]:
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [81]:
X = df.iloc[:, :-1] # explanatory variables

y = df.iloc[:,-1] #  target variable

In [82]:
y.value_counts()/len(y)

rented_last_month
False    0.999
True     0.001
Name: count, dtype: float64

In [93]:
numeric_features = ['rental_rate', 'length']

categorical_features = ['film_id', 'rating_G', 'rating_NC-17', 'rating_PG', 'rating_PG-13', 'rating_R']

In [94]:
numeric_transformer = Pipeline(steps = [("imputer", SimpleImputer(strategy="mean")),("scaler", StandardScaler())])

categorical_transformer = OneHotEncoder(drop = "first") 

In [95]:
preprocessor = ColumnTransformer(
            transformers = [("numerical", numeric_transformer, numeric_features),
                            ("categorical", categorical_transformer, categorical_features)]
)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state =42)

In [97]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

ValueError: Found unknown categories [3, 4, 11, 24, 26, 30, 31, 32, 34, 40, 45, 50, 55, 56, 60, 61, 64, 66, 67, 68, 71, 73, 77, 78, 79, 85, 87, 89, 97, 102, 108, 110, 111, 121, 137, 138, 140, 142, 159, 166, 169, 175, 185, 193, 199, 200, 205, 209, 210, 211, 214, 216, 219, 222, 232, 236, 238, 245, 248, 250, 255, 257, 260, 261, 262, 266, 276, 278, 281, 282, 287, 290, 293, 295, 297, 299, 300, 306, 307, 308, 310, 311, 313, 315, 319, 320, 322, 328, 329, 332, 333, 334, 345, 347, 351, 352, 356, 362, 364, 366, 371, 372, 378, 383, 409, 412, 426, 430, 431, 433, 437, 440, 446, 452, 463, 480, 482, 487, 494, 495, 496, 500, 514, 515, 522, 523, 527, 528, 529, 530, 535, 536, 537, 543, 544, 545, 549, 550, 555, 558, 560, 571, 576, 579, 584, 585, 590, 591, 596, 597, 600, 602, 605, 614, 618, 622, 627, 636, 637, 644, 645, 650, 651, 653, 656, 657, 661, 663, 665, 669, 671, 678, 679, 680, 688, 690, 693, 696, 698, 709, 714, 715, 717, 722, 735, 738, 741, 742, 750, 754, 762, 765, 768, 788, 800, 801, 803, 811, 812

In [98]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [99]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [100]:
pred_train = classifier.predict(X_train)
display(pred_train)

pred_test = classifier.predict(X_test)
display(pred_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'LIFE TWISTED'

In [101]:
array([0, 0, 0, ..., 0, 0, 0], dtype=int64)
array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

NameError: name 'array' is not defined

In [102]:
prob_train = classifier.predict_proba(X_train)

display(prob_train)

prob_test = classifier.predict_proba(X_test)
display(prob_test)

array([[9.98480821e-01, 1.51917870e-03],
       [9.98863848e-01, 1.13615220e-03],
       [9.97274822e-01, 2.72517773e-03],
       ...,
       [9.99567751e-01, 4.32249488e-04],
       [9.98234123e-01, 1.76587695e-03],
       [9.99217145e-01, 7.82854658e-04]])

c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'LIFE TWISTED'

In [103]:
print("accuracy on train set", accuracy_score(y_train, pred_train))
print("accuracy on test set", accuracy_score(y_test, pred_test))
print("f1 score on train set", f1_score(y_train, pred_train))
print("f1 score on test set", f1_score(y_test, pred_test))

accuracy on train set 0.9986666666666667


NameError: name 'pred_test' is not defined

In [104]:
print(classification_report(y_train, pred_train))

print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       749
           1       0.00      0.00      0.00         1

    accuracy                           1.00       750
   macro avg       0.50      0.50      0.50       750
weighted avg       1.00      1.00      1.00       750



c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rainm\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

NameError: name 'pred_test' is not defined